In [1]:
library(tsfeatures)
library(imputeTS)
library(forecast)
library(dplyr)

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


载入程辑包：‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
data=read.csv('GDP_worldbank.csv')

In [3]:
head(data)

,Country.Name,阿鲁巴,X,阿富汗,X.1,安哥拉,阿尔巴尼亚,安道尔共和国,阿拉伯联盟国家,阿拉伯联合酋长国,⋯,美属维京群岛,越南,瓦努阿图,世界,萨摩亚,科索沃,也门共和国,南非,赞比亚,津巴布韦
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1960,NA,21125015452,537777811,10447637853,NA,NA,NA,NA,NA,⋯,NA,NA,NA,1.38486e+12,NA,NA,NA,8748596504,713000000,1052990400
2,1961,NA,21616228139,548888896,11173212080,NA,NA,NA,NA,NA,⋯,NA,NA,NA,1.44922e+12,NA,NA,NA,9225996313,696285714,1096646600
3,1962,NA,23506279900,546666678,11990534018,NA,NA,NA,NA,NA,⋯,NA,NA,NA,1.55082e+12,NA,NA,NA,9813996078,693142857,1117601600
4,1963,NA,28048360188,751111191,12727688165,NA,NA,NA,NA,NA,⋯,NA,NA,NA,1.67145e+12,NA,NA,NA,10854195663,718714286,1159511700
5,1964,NA,25920665260,800000044,13898109284,NA,NA,NA,NA,NA,⋯,NA,NA,NA,1.83049e+12,NA,NA,NA,11955995222,839428571,1217138000
6,1965,NA,29472103270,1006666638,14929792388,NA,NA,NA,NA,NA,⋯,NA,NA,NA,1.99410e+12,NA,NA,NA,13068994778,1082857143,1311435800


In [4]:
tail(data)

,Country.Name,阿鲁巴,X,阿富汗,X.1,安哥拉,阿尔巴尼亚,安道尔共和国,阿拉伯联盟国家,阿拉伯联合酋长国,⋯,美属维京群岛,越南,瓦努阿图,世界,萨摩亚,科索沃,也门共和国,南非,赞比亚,津巴布韦
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
58,2017,3092178771,1.02104e+12,18896353156,6.84899e+11,68972769396,13019726212,3000162081,2.63711e+12,3.90517e+11,⋯,3.794e+09,2.81354e+11,880061831,8.14420e+13,884844420,7180768692,26842231205,3.81449e+11,25873601261,17584890937
59,2018,3276187709,1.00720e+12,18418860354,7.67026e+11,77792944472,15156424062,3218419897,2.84348e+12,4.27049e+11,⋯,3.922e+09,3.10106e+11,914736985,8.65022e+13,878448440,7878762812,21606161066,4.04159e+11,26311506435,34156069918
60,2019,3395793855,1.00083e+12,18904502222,8.22538e+11,69309110146,15401826081,3155149348,2.86889e+12,4.17990e+11,⋯,4.117e+09,3.34365e+11,936526268,8.77281e+13,912950619,7899741279,NA,3.88531e+11,23308667781,21832234926
61,2020,2610038939,9.27593e+11,20143451706,7.86460e+11,50241368244,15162734205,2891001149,2.53338e+12,3.49473e+11,⋯,4.204e+09,3.46616e+11,909421044,8.52152e+13,868898350,7717143395,NA,3.37620e+11,18110638619,21509698406
62,2021,3126019385,1.08200e+12,14583135237,8.44460e+11,65685435100,17930565119,3325145407,2.92846e+12,4.15022e+11,⋯,NA,3.66138e+11,971636098,9.68824e+13,843850778,9412034299,NA,4.19016e+11,22147649569,28371238666
63,2022,NA,1.16948e+12,NA,8.77863e+11,106714000000,18882095518,3352032737,3.55756e+12,5.07535e+11,⋯,NA,4.08802e+11,983582865,1.00562e+14,832421565,9429156202,NA,4.05870e+11,29784454056,20678055598


In [5]:
data=data[,c(2:dim(data)[2])]

In [6]:
dim(data)

[1]  63 266

In [7]:
which_baddata=function(data){
baddata=c()
for(i in 1:dim(data)[2])
{loc=min(which(is.na(data[,i])==FALSE))-1
if((dim(data)[1]-loc)<40)
    {
    baddata=append(baddata,i)
}
else if(sum(is.na(data[loc:dim(data)[1],i])==TRUE)>10)
 {
     baddata=append(baddata,i)
 }
else if(sum(is.na(data[(dim(data)[1]-11):dim(data)[1],i])==TRUE)>3)
 {
     baddata=append(baddata,i)
 }
}
baddata=unique(baddata)
return(baddata)}

In [8]:
baddata=which_baddata(data)

Warning message in min(which(is.na(data[, i]) == FALSE)):
“min里所有的参数都不存在; 回覆Inf”
Warning message in min(which(is.na(data[, i]) == FALSE)):
“min里所有的参数都不存在; 回覆Inf”
Warning message in min(which(is.na(data[, i]) == FALSE)):
“min里所有的参数都不存在; 回覆Inf”
Warning message in min(which(is.na(data[, i]) == FALSE)):
“min里所有的参数都不存在; 回覆Inf”


In [9]:
baddata

[1]   1   3   6  11  12  16  25  26  37  39  52  53  55  57  65  70  72  79  83
[20]  85  86  94 100 102 109 111 116 121 123 124 130 131 132 133 144 146 148 151
[39] 158 163 165 166 180 189 191 194 197 202 203 213 214 215 217 220 222 223 226
[58] 229 232 235 236 238 246 247 249 253 255 256 257 258 262 263

In [10]:
data=data[,-1*baddata]

In [11]:
data

X,X.1,安哥拉,安道尔共和国,阿拉伯联盟国家,阿拉伯联合酋长国,阿根廷,安提瓜和巴布达,澳大利亚,奥地利,⋯,中高等收入国家,乌拉圭,美国,圣文森特和格林纳丁斯,瓦努阿图,世界,萨摩亚,南非,赞比亚,津巴布韦
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
21125015452,10447637853,NA,NA,NA,NA,NA,NA,18606562979,6592693841,⋯,2.29314e+11,1242289239,5.43300e+11,13066634,NA,1.38486e+12,NA,8748596504,713000000,1052990400
21616228139,11173212080,NA,NA,NA,NA,NA,NA,19682883152,7311749633,⋯,2.17557e+11,1547388781,5.63300e+11,13999965,NA,1.44922e+12,NA,9225996313,696285714,1096646600
23506279900,11990534018,NA,NA,NA,NA,24450604878,NA,19922563190,7756110210,⋯,2.23123e+11,1710004407,6.05100e+11,14524964,NA,1.55082e+12,NA,9813996078,693142857,1117601600
28048360188,12727688165,NA,NA,NA,NA,18272123664,NA,21539843449,8374175258,⋯,2.34130e+11,1539681491,6.38600e+11,13708299,NA,1.67145e+12,NA,10854195663,718714286,1159511700
25920665260,13898109284,NA,NA,NA,NA,25605249382,NA,23801123811,9169983886,⋯,2.68661e+11,1975701816,6.85800e+11,14758296,NA,1.83049e+12,NA,11955995222,839428571,1217138000
29472103270,14929792388,NA,NA,NA,NA,28344705967,NA,25976164159,9994070616,⋯,3.00143e+11,1890767156,7.43700e+11,15108296,NA,1.99410e+12,NA,13068994778,1082857143,1311435800
32014368121,15910837742,NA,NA,NA,NA,28630474728,NA,27307844373,10887682273,⋯,3.31543e+11,1809185094,8.15000e+11,16099960,NA,2.16412e+12,NA,14211394321,1264285714,1281749500
33269509510,14510579889,NA,NA,NA,NA,24256667553,NA,30442724874,11579431669,⋯,3.33642e+11,1597713469,8.61700e+11,15835103,NA,2.30258e+12,NA,15821393678,1368000000,1397002000
36327785495,14968235782,NA,NA,34715404211,NA,26436857247,NA,32714085238,12440625313,⋯,3.50115e+11,1593674185,9.42500e+11,15350000,NA,2.48467e+12,NA,17124793157,1605857143,1479599900


In [12]:
freq=1
ll=dim(data)[2]
hh=6
start=c(1960,1)
m=5
n=4

In [13]:
process_data <- function(data, start, freq, hh, ll) {
  data_ts <- list()
  data_tstrain <- list()
  data_features <- NULL
  
  for (i in 1:ll) {
    # 寻找第一个非NA值的位置
    loc <- min(which(is.na(data[, i]) == FALSE)) - 1
    zc <- loc %/% freq
    yu <- loc %% freq
    
    # 提取子序列
    x <- data[(loc + 1):dim(data)[1], i]
    
    # 检测并处理NA值
    r <- which(is.na(x) == TRUE)
    if (length(r) != 0) {
      x <- na_kalman(x) # 假设na_kalman是一个处理NA值的函数
    }
    
    # 创建时间序列对象
    da <- ts(x[1:(length(x) - hh)], start = c(start[1] + zc, start[2] + yu), frequency = freq)
    da1 <- ts(x, start = c(start[1] + zc, start[2] + yu), frequency = freq)
    
    # 保存时间序列对象
    data_ts[[i]] <- da1
    data_tstrain[[i]] <- da
    
    # 提取时间序列特征
    f <- tsfeatures(da)
    if (is.null(data_features)) {
      data_features <- f
    } else {
      data_features <- rbind(data_features, f)
    }
  }
  
  # 函数返回一个列表，包含处理后的时间序列和特征
  list(time_series = data_ts, time_series_train = data_tstrain, features = data_features)
}


In [14]:
processed_data <- process_data(data, start = start, freq, hh, ll)

In [15]:
data_ts=processed_data$time_series
data_tstrain=processed_data$time_series_train
data_features=processed_data$features

In [16]:
head(data_features)

frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,1,0.9938539,5.561040e-08,6.525574,2.757101,0.5066125,0.5143639,0.3557599,0.9486600,3.996708,0.51362458,0.5403016,-0.2176705,0.5599868
1,0,1,0.9848451,6.162750e-07,6.026968,3.246783,0.3410315,0.3997375,0.3605710,0.9407587,3.435240,0.32612821,0.2002519,-0.3647270,0.3103642
1,0,1,0.9389677,1.936585e-05,4.482655,2.109987,0.4230610,0.4943193,0.5282683,0.9361755,2.722949,0.39919469,0.3415566,-0.2791351,0.4358000
1,0,1,0.9894822,2.686680e-07,6.132317,1.236822,0.4213672,0.5807627,0.3694987,0.9603655,4.622537,0.35364757,0.5697045,-0.3435493,0.4165376
1,0,1,0.9905445,1.781938e-07,6.017272,2.549240,0.2264496,0.2103130,0.2857718,0.9393229,3.665908,0.08571036,0.1642598,-0.5989873,0.6049262
1,0,1,0.9868473,4.599897e-07,5.600112,2.519136,0.2544355,0.5011100,0.3710992,0.9313581,3.519718,0.11275015,0.1678080,-0.4479392,0.4286456


In [17]:
long=matrix(0,ll,1)
for( i in 1:ll)
    {
    long[i,1]=length(data_tstrain[[i]])
}

In [18]:
data_features=cbind(data_features,long)

In [19]:
head(data_features)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,long
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,1,0.9938539,5.561040e-08,6.525574,2.757101,0.5066125,0.5143639,0.3557599,0.9486600,3.996708,0.51362458,0.5403016,-0.2176705,0.5599868,57
2,1,0,1,0.9848451,6.162750e-07,6.026968,3.246783,0.3410315,0.3997375,0.3605710,0.9407587,3.435240,0.32612821,0.2002519,-0.3647270,0.3103642,57
3,1,0,1,0.9389677,1.936585e-05,4.482655,2.109987,0.4230610,0.4943193,0.5282683,0.9361755,2.722949,0.39919469,0.3415566,-0.2791351,0.4358000,37
4,1,0,1,0.9894822,2.686680e-07,6.132317,1.236822,0.4213672,0.5807627,0.3694987,0.9603655,4.622537,0.35364757,0.5697045,-0.3435493,0.4165376,47
5,1,0,1,0.9905445,1.781938e-07,6.017272,2.549240,0.2264496,0.2103130,0.2857718,0.9393229,3.665908,0.08571036,0.1642598,-0.5989873,0.6049262,49
6,1,0,1,0.9868473,4.599897e-07,5.600112,2.519136,0.2544355,0.5011100,0.3710992,0.9313581,3.519718,0.11275015,0.1678080,-0.4479392,0.4286456,42


In [20]:
f1=function(x){
    if( x%%1>=0.5)
        {
        return(as.integer(x)+1)
    }
    else{return(as.integer(x))}
}

In [21]:
forecast_and_evaluate <- function(data_ts, data_tstrain, m,n,hh, ll) {
  predets <- array(0, dim = c(ll, m, n, 6))
  predhets <- array(0, dim = c(ll, m, n, hh))
  predthetaf <- array(0, dim = c(ll, m, n, 6))
  predhthetaf <- array(0, dim = c(ll, m, n, hh))
  predarima <- array(0, dim = c(ll, m, n, 6))
  predharima <- array(0, dim = c(ll, m, n, hh))
  prednnetar <- array(0, dim = c(ll, m, n, 6))
  predhnnetar <- array(0, dim = c(ll, m, n, hh))
  yreal <- list()
  
  for (k in 1:ll) {
    y <- data_tstrain[[k]]
    yall <- data_ts[[k]]
    y_pred <- ts(yall[(length(yall) - hh + 1):length(yall)], end = end(yall), frequency = frequency(yall))
    y_l=length(y)
    loc = 1:length(y)
    loc_m = as.integer(loc*m/length(y))
    filt_d = data.frame(y,loc_m)
    filt_0=filter(filt_d ,loc_m==0)
    m_l=count(filt_0)
    n_l=m_l/n
    m_l=as.integer(m_l)
    n_l=as.numeric(n_l)
    
    yreal[[k]] <- y_pred
    
    for (i in 0:(m - 1)) {
      for (j in 0:(n-1)) {
        Y <- ts(y[round((i*m_l+j*n_l)+1):y_l], end = end(y), frequency = frequency(yall))
        M <- ets(Y)
        pd <- forecast(M, h = hh)
        predhets[k, (i + 1), (j + 1), ] <- pd$mean
        predets[k, (i + 1), (j + 1), ] <- accuracy(pd, y_pred)[2, 1:6]
        
        M <- thetaf(Y, h = hh)
        pd <- forecast(M, h = hh)
        predhthetaf[k, (i + 1), (j + 1), ] <- pd$mean
        predthetaf[k, (i + 1), (j + 1), ] <- accuracy(pd, y_pred)[2, 1:6]
          
        M <- auto.arima(Y)
        pd <- forecast(M, h = hh)
        predharima[k, (i + 1), (j + 1), ] <- pd$mean
        predarima[k, (i + 1), (j + 1), ] <- accuracy(pd, y_pred)[2, 1:6]
          
        M <- nnetar(Y, h = hh)
        pd <- forecast(M, h = hh)
        predhnnetar[k, (i + 1), (j + 1), ] <- pd$mean
        prednnetar[k, (i + 1), (j + 1), ] <- accuracy(pd, y_pred)[2, 1:6]
        
      }
    }
  }
  
  list(
    pred_ets = list(predhets = predhets, predets = predets),
    pred_thetaf = list(predhthetaf = predhthetaf, predthetaf = predthetaf),
    pred_arima = list(predharima = predharima, predarima = predarima),
    pred_nnetar = list(predhnnetar = predhnnetar, prednnetar = prednnetar),
    y_real = yreal
  )
}

In [22]:
forecast_list=forecast_and_evaluate(data_ts, data_tstrain, m,n,hh, ll)

In [ ]:
predhets=forecast_list$pred_ets$predhets
predets=forecast_list$pred_ets$predets
predhthetaf=forecast_list$pred_thetaf$predhthetaf
predthetaf=forecast_list$pred_thetaf$predthetaf

predharima=forecast_list$pred_arima$predharima
predarima=forecast_list$pred_arima$predarima
predhnnetar=forecast_list$pred_nnetar$predhnnetar
prednnetar=forecast_list$pred_nnetar$prednnetar

y_real=forecast_list$y_real

In [ ]:
length5=matrix(0,ll,m)
for(k in 1:ll)
{
y = data_tstrain[[k]]
y_l=length(y)
loc = 1:length(y)
loc_m = as.integer(loc*m/length(y))
filt_d = data.frame(y,loc_m)
filt_0=filter(filt_d ,loc_m==0)
m_l=count(filt_0)
m_l=as.integer(m_l)
length5[k,]=c(0,1*m_l,2*m_l,3*m_l,4*m_l)
    }

In [ ]:
library("xgboost")
library("Matrix")
library('Ckmeans.1d.dp')
library(lightgbm)

In [26]:
meanunique=function(x)
    {
    mean(unique(x))
}

In [27]:
whichmin<-function(x){
    minx=min(x[x>0])
    loc=which(x==minx)[1]-1
    loc
}

In [28]:
realbest_construct<-function(MASE)
    {
realbestmean=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    line=apply(count,1,meanunique)
    if (max(line,na.rm = TRUE)==0){
        realbestmean[i,]=0
        }
    else
        {
        line[is.na(line)]=100 
        realbestmean[i,]= whichmin(line)
    }
    }
    return(realbestmean)
    }
    

In [29]:
MASEets=predets[,,,6]
MASEthetaf=predthetaf[,,,6]
MASEarima=predarima[,,,6]
MASEnnetar=prednnetar[,,,6]

realbestmeanets=realbest_construct(MASEets)
realbestmeanthetaf=realbest_construct(MASEthetaf)
realbestmeanarima=realbest_construct(MASEarima)
realbestmeannnetar=realbest_construct(MASEnnetar)

In [30]:
set.seed(50)
index = sample(2,nrow(data_features),replace = TRUE,prob=c(0.7,0.3))
train_data=data_features[index==1,]
test_data=data_features[index==2,]
train_label_ets=realbestmeanets[index==1,]
test_label_ets=realbestmeanets[index==2,]
train_label_thetaf=realbestmeanthetaf[index==1,]
test_label_thetaf=realbestmeanthetaf[index==2,]
               
train_label_arima=realbestmeanarima[index==1,]
test_label_arima=realbestmeanarima[index==2,]
train_label_nnetar=realbestmeannnetar[index==1,]
test_label_nnetar=realbestmeannnetar[index==2,]

## 模型训练

In [31]:
train_models<-function(train_data,train_label)
{
    dtrain_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label)) 
    dtrain_cl<- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label)))
    xgbreg  <- xgboost(data = dtrain_reg, nround=100)
    xgbcls  <- xgboost(data = dtrain_cl, nround=100, objective='multi:softmax',num_class=5)
    dtrain_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label))
    dtrain_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label)))
    # 定义参数列表
    params_reg <- list(
      objective = "regression",
      metric = "rmse"
    )
    lgbreg  <- lgb.train(data = dtrain_reg,nrounds = 100,params = params_reg)
    # 定义参数列表
    params_cl <- list(
      objective = 'multiclass',
      num_class = 5,
      num_iterations = 100  # 使用 num_iterations 代替 nrounds
    )
    # 使用参数列表进行训练
    lgbcls <- lgb.train(data = dtrain_cl,params = params_cl)
    return(list(xgbreg =xgbreg, xgbcls = xgbcls,lgbreg=lgbreg,lgbcls=lgbcls))
}    

In [32]:
models_ets=train_models(train_data,train_label_ets)
etsxgbreg=models_ets$xgbreg
etsxgbcls=models_ets$xgbcls
etslgbreg=models_ets$lgbreg
etslgbcls=models_ets$lgbcls

[1]	train-rmse:1.950327 
[2]	train-rmse:1.463520 
[3]	train-rmse:1.127992 
[4]	train-rmse:0.861637 
[5]	train-rmse:0.677288 
[6]	train-rmse:0.544908 
[7]	train-rmse:0.439676 
[8]	train-rmse:0.365571 
[9]	train-rmse:0.308176 
[10]	train-rmse:0.262430 
[11]	train-rmse:0.235345 
[12]	train-rmse:0.205735 
[13]	train-rmse:0.187597 
[14]	train-rmse:0.154592 
[15]	train-rmse:0.137919 
[16]	train-rmse:0.117626 
[17]	train-rmse:0.105190 
[18]	train-rmse:0.091000 
[19]	train-rmse:0.080918 
[20]	train-rmse:0.074619 
[21]	train-rmse:0.066101 
[22]	train-rmse:0.056915 
[23]	train-rmse:0.050129 
[24]	train-rmse:0.042681 
[25]	train-rmse:0.036371 
[26]	train-rmse:0.033471 
[27]	train-rmse:0.027529 
[28]	train-rmse:0.023026 
[29]	train-rmse:0.019493 
[30]	train-rmse:0.016953 
[31]	train-rmse:0.014143 
[32]	train-rmse:0.012130 
[33]	train-rmse:0.010896 
[34]	train-rmse:0.010201 
[35]	train-rmse:0.008415 
[36]	train-rmse:0.007548 
[37]	train-rmse:0.006613 
[38]	train-rmse:0.005643 
[39]	train-rmse:0.004

In [33]:
models_thetaf=train_models(train_data,train_label_thetaf)
thetafxgbreg=models_thetaf$xgbreg
thetafxgbcls=models_thetaf$xgbcls
thetaflgbreg=models_thetaf$lgbreg
thetaflgbcls=models_thetaf$lgbcls

[1]	train-rmse:2.062109 
[2]	train-rmse:1.483040 
[3]	train-rmse:1.079106 
[4]	train-rmse:0.798414 
[5]	train-rmse:0.591206 
[6]	train-rmse:0.444666 
[7]	train-rmse:0.336368 
[8]	train-rmse:0.263990 
[9]	train-rmse:0.206961 
[10]	train-rmse:0.161609 
[11]	train-rmse:0.127663 
[12]	train-rmse:0.102639 
[13]	train-rmse:0.083199 
[14]	train-rmse:0.069345 
[15]	train-rmse:0.058498 
[16]	train-rmse:0.048090 
[17]	train-rmse:0.040597 
[18]	train-rmse:0.034624 
[19]	train-rmse:0.030116 
[20]	train-rmse:0.025641 
[21]	train-rmse:0.022648 
[22]	train-rmse:0.019872 
[23]	train-rmse:0.016658 
[24]	train-rmse:0.014690 
[25]	train-rmse:0.012896 
[26]	train-rmse:0.011393 
[27]	train-rmse:0.009800 
[28]	train-rmse:0.009020 
[29]	train-rmse:0.007644 
[30]	train-rmse:0.006602 
[31]	train-rmse:0.005986 
[32]	train-rmse:0.004972 
[33]	train-rmse:0.004299 
[34]	train-rmse:0.003724 
[35]	train-rmse:0.003208 
[36]	train-rmse:0.002681 
[37]	train-rmse:0.002404 
[38]	train-rmse:0.002104 
[39]	train-rmse:0.001

In [34]:
models_arima=train_models(train_data,train_label_arima)
arimaxgbreg=models_arima$xgbreg
arimaxgbcls=models_arima$xgbcls
arimalgbreg=models_arima$lgbreg
arimalgbcls=models_arima$lgbcls

[1]	train-rmse:1.828005 
[2]	train-rmse:1.356376 
[3]	train-rmse:1.012823 
[4]	train-rmse:0.769659 
[5]	train-rmse:0.605089 
[6]	train-rmse:0.473702 
[7]	train-rmse:0.388454 
[8]	train-rmse:0.323210 
[9]	train-rmse:0.267018 
[10]	train-rmse:0.225197 
[11]	train-rmse:0.187977 
[12]	train-rmse:0.163493 
[13]	train-rmse:0.146127 
[14]	train-rmse:0.123908 
[15]	train-rmse:0.104863 
[16]	train-rmse:0.089074 
[17]	train-rmse:0.073812 
[18]	train-rmse:0.066320 
[19]	train-rmse:0.058661 
[20]	train-rmse:0.052955 
[21]	train-rmse:0.045705 
[22]	train-rmse:0.040569 
[23]	train-rmse:0.034220 
[24]	train-rmse:0.029003 
[25]	train-rmse:0.023819 
[26]	train-rmse:0.020933 
[27]	train-rmse:0.017195 
[28]	train-rmse:0.014474 
[29]	train-rmse:0.012702 
[30]	train-rmse:0.011311 
[31]	train-rmse:0.010385 
[32]	train-rmse:0.009144 
[33]	train-rmse:0.008609 
[34]	train-rmse:0.007929 
[35]	train-rmse:0.007630 
[36]	train-rmse:0.006483 
[37]	train-rmse:0.006007 
[38]	train-rmse:0.005348 
[39]	train-rmse:0.004

In [35]:
models_nnetar=train_models(train_data,train_label_nnetar)
nnetarxgbreg=models_nnetar$xgbreg
nnetarxgbcls=models_nnetar$xgbcls
nnetarlgbreg=models_nnetar$lgbreg
nnetarlgbcls=models_nnetar$lgbcls

[1]	train-rmse:2.146792 
[2]	train-rmse:1.545651 
[3]	train-rmse:1.129240 
[4]	train-rmse:0.838614 
[5]	train-rmse:0.629446 
[6]	train-rmse:0.479084 
[7]	train-rmse:0.377899 
[8]	train-rmse:0.297881 
[9]	train-rmse:0.234795 
[10]	train-rmse:0.191781 
[11]	train-rmse:0.164734 
[12]	train-rmse:0.141258 
[13]	train-rmse:0.124131 
[14]	train-rmse:0.105246 
[15]	train-rmse:0.097834 
[16]	train-rmse:0.086124 
[17]	train-rmse:0.076818 
[18]	train-rmse:0.067494 
[19]	train-rmse:0.058839 
[20]	train-rmse:0.052625 
[21]	train-rmse:0.047773 
[22]	train-rmse:0.040883 
[23]	train-rmse:0.036501 
[24]	train-rmse:0.034093 
[25]	train-rmse:0.030184 
[26]	train-rmse:0.026669 
[27]	train-rmse:0.022321 
[28]	train-rmse:0.020249 
[29]	train-rmse:0.019025 
[30]	train-rmse:0.017207 
[31]	train-rmse:0.015603 
[32]	train-rmse:0.014125 
[33]	train-rmse:0.012681 
[34]	train-rmse:0.011284 
[35]	train-rmse:0.009872 
[36]	train-rmse:0.008884 
[37]	train-rmse:0.007391 
[38]	train-rmse:0.006593 
[39]	train-rmse:0.006

## 预测与输出误差

In [36]:
alldatalgb <- lgb.Dataset(data = as.matrix(data_features))
alldataxgb <- xgb.DMatrix(data = as.matrix(data_features))

In [37]:
lgbcls_change<-function(lgbcls,ll)
    {
    lgbclsm=matrix(lgbcls,5,ll)
    lgbclsr=matrix(0,ll,1)
    for(i in 1:ll)
        {
        lgbclsr[i,]=which.max(lgbclsm[,i])

    }
    return(lgbclsr)
    }

In [38]:
xgbregets=predict(etsxgbreg,alldataxgb)
xgbclsets=predict(etsxgbcls,alldataxgb)
lgbregets=predict(etslgbreg,as.matrix(data_features))
lgbclsets=predict(etslgbcls,as.matrix(data_features))

In [39]:
xgbregthetaf=predict(thetafxgbreg,alldataxgb)
xgbclsthetaf=predict(thetafxgbcls,alldataxgb)
lgbregthetaf=predict(thetaflgbreg,as.matrix(data_features))
lgbclsthetaf=predict(thetaflgbcls,as.matrix(data_features))

In [40]:
xgbregarima=predict(arimaxgbreg,alldataxgb)
xgbclsarima=predict(arimaxgbcls,alldataxgb)
lgbregarima=predict(arimalgbreg,as.matrix(data_features))
lgbclsarima=predict(arimalgbcls,as.matrix(data_features))

In [41]:
xgbregnnetar=predict(nnetarxgbreg,alldataxgb)
xgbclsnnetar=predict(nnetarxgbcls,alldataxgb)
lgbregnnetar=predict(nnetarlgbreg,as.matrix(data_features))
lgbclsnnetar=predict(nnetarlgbcls,as.matrix(data_features))

In [42]:
lgbclsetsr=lgbcls_change(lgbclsets,ll)
lgbclsthetafr=lgbcls_change(lgbclsthetaf,ll)
lgbclsarimar=lgbcls_change(lgbclsarima,ll)
lgbclsnnetarr=lgbcls_change(lgbclsnnetar,ll)

In [43]:
round5=function(x)
    {
    l=round(x)
    for(i in 1:length(l))
    {
    if(l[i]>5)
        {
        l[i]=5
    }
    if(l[i]<1)
    {
        l[i]=1
    }
}
    l
}

In [44]:
ets_pr=matrix(0,ll,4)
ets_pr[,1]=round5(xgbregets+1)
ets_pr[,2]=xgbclsets+1
ets_pr[,3]=round5(lgbregets+1)
ets_pr[,4]=lgbclsetsr

In [45]:
thetaf_pr=matrix(0,ll,4)
thetaf_pr[,1]=round5(xgbregthetaf+1)
thetaf_pr[,2]=xgbclsthetaf+1
thetaf_pr[,3]=round5(lgbregthetaf+1)
thetaf_pr[,4]=lgbclsthetafr

In [46]:
arima_pr=matrix(0,ll,4)
arima_pr[,1]=round5(xgbregarima+1)
arima_pr[,2]=xgbclsarima+1
arima_pr[,3]=round5(lgbregarima+1)
arima_pr[,4]=lgbclsarimar

In [47]:
nnetar_pr=matrix(0,ll,4)
nnetar_pr[,1]=round5(xgbregnnetar+1)
nnetar_pr[,2]=xgbclsthetaf+1
nnetar_pr[,3]=round5(lgbregnnetar+1)
nnetar_pr[,4]=lgbclsnnetarr

In [48]:
meanpre=function(pred_array,opt_pr,h,testindex)
    {
    data_length=length(testindex)
    meanpreh=matrix(0,data_length,h)
    for (i in 1:data_length){
    index=testindex[i]
    loc=opt_pr[index]
    predh=pred_array[index,loc,,]
    predh=unique(predh)
    meanpreh[i,]=apply(predh,2,mean)
}
    return(meanpreh)
}

In [49]:
predres=function(pred_matrix,data,real_data,opt_pr,lengthmatrix,h,testindex)
    {
    data_length=length(testindex)
    pred_res=matrix(0,data_length,6)
    model=ets(data[[1]])
    fore_l=forecast(model,h=h)
    freq=frequency(data[[1]])
    for (i in 1:data_length){
    index=testindex[i]
    loc=opt_pr[index]
    start=lengthmatrix[i,loc]
    y_all=data[[index]]
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=real_data[[index]]
    fore_l$mean=ts(pred_matrix[i,],start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    pred_res[i,]=res[2,1:6]
    }
    return(pred_res)
    }

In [50]:
testindex=c(1:ll)[index==2]
lengthtest=length5[testindex,]
etsxgbcls=meanpre(predhets,ets_pr[,1],hh,testindex)
etsxgbreg=meanpre(predhets,ets_pr[,2],hh,testindex)
etslgbcls=meanpre(predhets,ets_pr[,3],hh,testindex)
etslgbreg=meanpre(predhets,ets_pr[,4],hh,testindex)
etsxgbclsres=predres(etsxgbcls,data_tstrain,y_real,ets_pr[,1],lengthtest,hh,testindex)
etsxgbregres=predres(etsxgbreg,data_tstrain,y_real,ets_pr[,2],lengthtest,hh,testindex)
etslgbclsres=predres(etslgbcls,data_tstrain,y_real,ets_pr[,3],lengthtest,hh,testindex)
etslgbregres=predres(etslgbreg,data_tstrain,y_real,ets_pr[,4],lengthtest,hh,testindex)

In [51]:
thetafxgbcls=meanpre(predhthetaf,thetaf_pr[,1],hh,testindex)
thetafxgbreg=meanpre(predhthetaf,thetaf_pr[,2],hh,testindex)
thetaflgbcls=meanpre(predhthetaf,thetaf_pr[,3],hh,testindex)
thetaflgbreg=meanpre(predhthetaf,thetaf_pr[,4],hh,testindex)
thetafxgbclsres=predres(thetafxgbcls,data_tstrain,y_real,thetaf_pr[,1],lengthtest,hh,testindex)
thetafxgbregres=predres(thetafxgbreg,data_tstrain,y_real,thetaf_pr[,2],lengthtest,hh,testindex)
thetaflgbclsres=predres(thetaflgbcls,data_tstrain,y_real,thetaf_pr[,3],lengthtest,hh,testindex)
thetaflgbregres=predres(thetaflgbreg,data_tstrain,y_real,thetaf_pr[,4],lengthtest,hh,testindex)

In [52]:
arimaxgbcls=meanpre(predharima,arima_pr[,1],hh,testindex)
arimaxgbreg=meanpre(predharima,arima_pr[,2],hh,testindex)
arimalgbcls=meanpre(predharima,arima_pr[,3],hh,testindex)
arimalgbreg=meanpre(predharima,arima_pr[,4],hh,testindex)
arimaxgbclsres=predres(arimaxgbcls,data_tstrain,y_real,ets_pr[,1],lengthtest,hh,testindex)
arimaxgbregres=predres(arimaxgbreg,data_tstrain,y_real,ets_pr[,2],lengthtest,hh,testindex)
arimalgbclsres=predres(arimalgbcls,data_tstrain,y_real,ets_pr[,3],lengthtest,hh,testindex)
arimalgbregres=predres(arimalgbreg,data_tstrain,y_real,ets_pr[,4],lengthtest,hh,testindex)

In [53]:
nnetarxgbcls=meanpre(predhnnetar,nnetar_pr[,1],hh,testindex)
nnetarxgbreg=meanpre(predhnnetar,nnetar_pr[,2],hh,testindex)
nnetarlgbcls=meanpre(predhnnetar,nnetar_pr[,3],hh,testindex)
nnetarlgbreg=meanpre(predhnnetar,nnetar_pr[,4],hh,testindex)
nnetarxgbclsres=predres(nnetarxgbcls,data_tstrain,y_real,nnetar_pr[,1],lengthtest,hh,testindex)
nnetarxgbregres=predres(nnetarxgbreg,data_tstrain,y_real,nnetar_pr[,2],lengthtest,hh,testindex)
nnetarlgbclsres=predres(nnetarlgbcls,data_tstrain,y_real,nnetar_pr[,3],lengthtest,hh,testindex)
nnetarlgbregres=predres(nnetarlgbreg,data_tstrain,y_real,nnetar_pr[,4],lengthtest,hh,testindex)

In [54]:
apply(predets[,1,1,],2,mean)
apply(etsxgbclsres,2,mean)
apply(etsxgbregres,2,mean)
apply(etslgbclsres,2,mean)
apply(etslgbregres,2,mean)

[1] 5.163549e+11 5.976835e+11 5.273158e+11 8.390187e+00 1.511252e+01
[6] 5.107665e+00

[1] 3.259605e+11 4.261670e+11 3.647779e+11 5.905189e+00 1.205264e+01
[6] 2.163172e+00

[1] 3.151213e+11 4.148822e+11 3.540182e+11 4.631779e+00 1.131940e+01
[6] 2.032850e+00

[1] 3.579900e+11 4.441479e+11 3.824431e+11 6.689208e+00 1.161060e+01
[6] 2.175576e+00

[1] 5.418921e+11 6.351584e+11 5.575361e+11 1.204435e+01 1.613865e+01
[6] 3.594504e+00

In [55]:
apply(predthetaf[,1,1,],2,mean)
apply(thetafxgbclsres,2,mean)
apply(thetafxgbregres,2,mean)
apply(thetaflgbclsres,2,mean)
apply(thetaflgbregres,2,mean)

[1] 4.806380e+11 5.637302e+11 4.963422e+11 8.087625e+00 1.343133e+01
[6] 5.168254e+00

[1] 3.797311e+11 4.653175e+11 4.025784e+11 3.752456e+00 1.116870e+01
[6] 2.007279e+00

[1] 3.809066e+11 4.667529e+11 4.039621e+11 3.498630e+00 1.106886e+01
[6] 2.018449e+00

[1] 3.793238e+11 4.633085e+11 4.008680e+11 5.130446e+00 1.102886e+01
[6] 1.984673e+00

[1] 4.440328e+11 5.375353e+11 4.678736e+11 5.715390e+00 1.221795e+01
[6] 3.178440e+00

In [56]:
apply(predarima[,1,1,],2,mean)
apply(arimaxgbclsres,2,mean)
apply(arimaxgbregres,2,mean)
apply(arimalgbclsres,2,mean)
apply(arimalgbregres,2,mean)

[1] 5.007497e+11 5.916383e+11 5.181969e+11 5.482727e+00 1.261836e+01
[6] 4.549950e+00

[1] 4.929055e+11 5.853623e+11 5.114332e+11 6.575515e+00 1.206644e+01
[6] 2.159845e+00

[1] 5.793992e+11 6.693775e+11 5.901769e+11 7.706192e+00 1.323474e+01
[6] 2.403580e+00

[1] 6.402621e+11 7.230763e+11 6.466159e+11 4.110918e+00 1.612938e+01
[6] 2.503574e+00

[1] 7.152078e+11 7.908279e+11 7.194663e+11 1.049075e+01 1.519368e+01
[6] 3.685295e+00

In [57]:
apply(prednnetar[,1,1,],2,mean)
apply(nnetarxgbclsres,2,mean)
apply(nnetarxgbregres,2,mean)
apply(nnetarlgbclsres,2,mean)
apply(nnetarlgbregres,2,mean)

[1] 4.868663e+11 5.956023e+11 5.186000e+11 6.312613e+00 1.473511e+01
[6] 5.589349e+00

[1] 5.762998e+11 6.763923e+11 5.916691e+11 6.890092e+00 1.382314e+01
[6] 2.449211e+00

[1] 5.381064e+11 6.401532e+11 5.521458e+11 5.746072e+00 1.431768e+01
[6] 2.632709e+00

[1] 5.769644e+11 6.752278e+11 5.914225e+11 6.916699e+00 1.396756e+01
[6] 2.559386e+00

[1] 5.752382e+11 6.746420e+11 5.906808e+11 4.745535e+00 1.434174e+01
[6] 3.463177e+00

In [59]:
alltest=matrix(0,20,6)
alltest[1,]=apply(predets[,1,1,],2,mean)
alltest[2,]=apply(etsxgbclsres,2,mean)
alltest[3,]=apply(etsxgbregres,2,mean)
alltest[4,]=apply(etslgbclsres,2,mean)
alltest[5,]=apply(etslgbregres,2,mean)

alltest[6,]=apply(predthetaf[,1,1,],2,mean)
alltest[7,]=apply(thetafxgbclsres,2,mean)
alltest[8,]=apply(thetafxgbregres,2,mean)
alltest[9,]=apply(thetaflgbclsres,2,mean)
alltest[10,]=apply(thetaflgbregres,2,mean)

alltest[11,]=apply(predarima[,1,1,],2,mean)
alltest[12,]=apply(arimaxgbclsres,2,mean)
alltest[13,]=apply(arimaxgbregres,2,mean)
alltest[14,]=apply(arimalgbclsres,2,mean)
alltest[15,]=apply(arimalgbregres,2,mean)

alltest[16,]=apply(prednnetar[,1,1,],2,mean)
alltest[17,]=apply(nnetarxgbclsres,2,mean)
alltest[18,]=apply(nnetarxgbregres,2,mean)
alltest[19,]=apply(nnetarlgbclsres,2,mean)
alltest[20,]=apply(nnetarlgbregres,2,mean)



In [60]:
write.csv(alltest,'GDP_by_oneself_12.csv')